In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import os

In [2]:
tavily_api_key = os.getenv('TAVILY_API_KEY')

os.environ['TAVILY_API_KEY'] = tavily_api_key

In [6]:
def tavily_data(query: str):
    try:
        tool = TavilySearchResults(
            max_results=5,
            include_domains=[
                "https://crypto.news/",
                "https://cointelegraph.com/",
                "https://dexscreener.com/"
            ],
            include_images=False,
            include_videos=False,
            include_links=True
        )
        results = tool.invoke(query)

        if not isinstance(results, list):
            raise ValueError("Unexpected result format: Expected a list.")

        filtered_results = [
            {"title": item.get("title", "No Title"), "content": item.get("content", "No Content")}
            for item in results
        ]

        return filtered_results

    except Exception as e:
        print(f"Error occurred in tavily_data: {e}")
        return results

In [7]:
tavily_data("sldjkfhlksdhgiosuhvnspiohjtiojmovmsd[dmf0p[]osdjgjuioshdfiosdjfpsd[djgsdohnfgjksdjfmsdopmfvosjngpsdoid]]")

[]